In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import gensim

from tqdm.auto import tqdm
tqdm.pandas()

from unicode import join_jamos

c:\python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_TYPE = 'full-morphs-jamo'
DATA_NOSTOPS_TYPE = 'full-nouns-jamo'

In [3]:
DATA_DIR = 'lol'
TOKENIZING_DIR = '3-tokenizing'
EMBEDDING_DIR = '4-embedding'

tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
embedding_dir = Path(EMBEDDING_DIR) / DATA_DIR
embedding_dir.mkdir(parents=True, exist_ok=True)

### tokenize

In [ ]:
df = pd.read_csv(tokenizing_dir / 'news-{}.csv'.format(DATA_TYPE)).dropna()
contents = [x.split(' ') for x in df.contents]

### model

In [ ]:
bigram = gensim.models.Phrases(contents)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[contents])
trigram_mod = gensim.models.phrases.Phraser(trigram)

### save

In [ ]:
bigram_mod_path = embedding_dir / 'news-{}-bigram.bin'.format(DATA_TYPE)
bigram_mod.save(str(bigram_mod_path))

trigram_mod_path = embedding_dir / 'news-{}-trigram.bin'.format(DATA_TYPE)
trigram_mod.save(str(trigram_mod_path))

### load

In [4]:
bigram_mod_path = embedding_dir / 'news-{}-bigram.bin'.format(DATA_TYPE)
trigram_mod_path = embedding_dir / 'news-{}-trigram.bin'.format(DATA_TYPE)

bigram_mod = gensim.models.phrases.Phraser.load(str(bigram_mod_path))
trigram_mod = gensim.models.phrases.Phraser.load(str(trigram_mod_path))

def make_bigrams(texts):
    return [bigram_mod[doc] if doc is not np.nan else doc for doc in tqdm(texts)]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] if doc is not np.nan else doc for doc in tqdm(texts)]

### transform

In [5]:
ndf = pd.read_csv(tokenizing_dir / 'news-{}.csv'.format(DATA_NOSTOPS_TYPE))

title_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.title])
lede_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.lede])
contents_trigrams = make_trigrams([x.split(' ') if x is not np.nan else x for x in ndf.contents])

ndf.title = [' '.join(x) if x is not np.nan else x for x in title_trigrams]
ndf.lede = [' '.join(x) if x is not np.nan else x for x in title_trigrams]
ndf.contents = [' '.join(x) if x is not np.nan else x for x in title_trigrams]

In [7]:
# bigram
[join_jamos(x) for x in ndf.title[1000:1010]]

['진에어 갱_플랭크 신경전',
 '진에어 년 테크_트리',
 '롤_드 컵 선발전 갱_맘 이창석 분위기_반전 진에어 세트 승리',
 '롤_드 컵 선발전 야스오 운영 세트',
 '롤_드 컵 선발전 마지막 교전 방 역전승 세트 선취',
 '롤_드 컵 선발전 진에어 날개 우승 드_컵 숙원',
 '전투_민족 인_빅 터스 롤_드 컵 진출 번 팀',
 '감독 코멘트 이지훈_감독 팀 시너지 발휘',
 '팀 창단 후 롤_드 컵 진출 성공',
 '롤_드 컵 선발전 썸데이 김찬호 야스오 활약 세트 승리']

In [7]:
# trigram
[join_jamos(x) for x in ndf.title[1000:1010]]

['진에어 갱_플랭크 신경전',
 '진에어 년 테크_트리',
 '롤_드_컵 선발전 갱_맘 이창석 분위기_반전 진에어 세트 승리',
 '롤_드_컵 선발전 야스오 운영 세트',
 '롤_드_컵 선발전 마지막 교전 방 역전승 세트 선취',
 '롤_드_컵 선발전 진에어 날개 우승 드_컵 숙원',
 '전투_민족 인_빅_터스 롤_드_컵 진출 번 팀',
 '감독 코멘트 이지훈_감독 팀 시너지 발휘',
 '팀 창단_후 롤_드_컵 진출 성공',
 '롤_드_컵 선발전 썸데이 김찬호 야스오 활약 세트 승리']

In [7]:
ndf.to_csv(embedding_dir / 'news-{}-trigram.csv'.format(DATA_NOSTOPS_TYPE), index=False)
ndf

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 12:49,20150801 12:49,1,1,ㅎㅏㅅ_ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷㅐㅂ...,ㅎㅏㅅ_ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷㅐㅂ...,인벤,ㅎㅏㅅ_ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷㅐㅂ...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄴㅠ_ㅋㅡㄹㄹㅣㅇㅓ ㅇㅣㄴㅌㅓㅂㅠ ㅇㅏㅍ ㅅㅡㅇㄹㅣ,ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄴㅠ_ㅋㅡㄹㄹㅣㅇㅓ ㅇㅣㄴㅌㅓㅂㅠ ㅇㅏㅍ ㅅㅡㅇㄹㅣ,헝그리앱,ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄴㅠ_ㅋㅡㄹㄹㅣㅇㅓ ㅇㅣㄴㅌㅓㅂㅠ ㅇㅏㅍ ㅅㅡㅇㄹㅣ,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,ㅅㅓㅁㅁㅓ ㅂㅐㅇ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ_ㄹㅐㅇㅋㅡ ㄷㅔ ㅁㅔㄴㅌ...,ㅅㅓㅁㅁㅓ ㅂㅐㅇ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ_ㄹㅐㅇㅋㅡ ㄷㅔ ㅁㅔㄴㅌ...,헝그리앱,ㅅㅓㅁㅁㅓ ㅂㅐㅇ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ_ㄹㅐㅇㅋㅡ ㄷㅔ ㅁㅔㄴㅌ...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗㅇㅇ...,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗㅇㅇ...,헝그리앱,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗㅇㅇ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㅌㅏ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅅㅔㅌㅡ ㅅㅡㅇㄹㅣ,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㅌㅏ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅅㅔㅌㅡ ㅅㅡㅇㄹㅣ,헝그리앱,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ_ㅅㅗㄴㅣㄱㅂㅜㅁ ㅌㅏ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅅㅔㅌㅡ ㅅㅡㅇㄹㅣ,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㄷㅏㅇㅣㅂㅡ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅔㅌ...,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㄷㅏㅇㅣㅂㅡ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅔㅌ...,인벤,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㄷㅏㅇㅣㅂㅡ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅔㅌ...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ_ㄹㅏㅇㅣㄴㅇㅓㅂ ...,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ_ㄹㅏㅇㅣㄴㅇㅓㅂ ...,데일리e스포츠,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ_ㄹㅏㅇㅣㄴㅇㅓㅂ ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ_ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ_...,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ_ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ_...,엑스포츠뉴스,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ_ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ_...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅋㅏㅇㅜㄴㅌㅓ ㄱㅣㅅㅓㄴ_ㅈㅔㅇㅏㅂ,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅋㅏㅇㅜㄴㅌㅓ ㄱㅣㅅㅓㄴ_ㅈㅔㅇㅏㅂ,OSEN,ㅅㅡㅇㄱㅏㅇ_ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅋㅏㅇㅜㄴㅌㅓ ㄱㅣㅅㅓㄴ_ㅈㅔㅇㅏㅂ,0.0,490
